In [ ]:
from RTSR3 import *
import matplotlib.pyplot as plt

rtsr3 = RTSR3(2, 32)
# Show params
params_info = paddle.summary(rtsr3,(1, 3, 180, 390))
print(params_info)
custom_dataset = SrDataset('../data/540cut','../data/1080cut')
print('-'*75,'\ncustom_dataset images: ',len(custom_dataset))

# ÓÃ DataLoader ÊµÏÖÊý¾Ý¼ÓÔØ
train_loader = paddle.io.DataLoader(custom_dataset, batch_size=1, shuffle=False)
# print("[Dataloader] Done!")

# ½«rtsr3Ä£ÐÍ¼°ÆäËùÓÐ×Ó²ãÉèÖÃÎªÑµÁ·Ä£Ê½¡£ÕâÖ»»áÓ°ÏìÄ³Ð©Ä£¿é£¬ÈçDropoutºÍBatchNorm¡£
rtsr3.train()
# print("[Training] Setting done!")

# ÉèÖÃµü´ú´ÎÊý
epochs = 10

# ÉèÖÃÓÅ»¯Æ÷
optim = paddle.optimizer.Adam(parameters=rtsr3.parameters())
# ÉèÖÃËðÊ§º¯Êý
loss_fn = paddle.nn.MSELoss()
acc_fn = Precision()
# set hisfea0, hisfea1
hisfea0 = None
hisfea1 = None
tag0 = 0
tag1 = 0
losslist = []
acclist = []
# acc_fn = paddle.metric.accuracy()
for epoch in range(epochs):
    for batch_id, data in enumerate(train_loader()):
        # print("[Training] Date fetching done!")
        x_data = data[0]            # ÑµÁ·Êý¾Ý
        y_data = data[1]            # ÑµÁ·Êý¾Ý±êÇ©
        # print(batch_id, x_data.shape, y_data.shape)
        predicts, hisfea0, hisfea1, tag0, tag1 = rtsr3(x_data, hisfea0, hisfea1, tag0, tag1)    # Ô¤²â½á¹û
        hisfea0.stop_gradient = True
        hisfea1.stop_gradient = True
        # ¼ÆËãËðÊ§ µÈ¼ÛÓÚ prepare ÖÐlossµÄÉèÖÃ
        loss = loss_fn(predicts, y_data)
        # # ¼ÆËã×¼È·ÂÊ µÈ¼ÛÓÚ prepare ÖÐmetricsµÄÉèÖÃ
        acc_fn.update(predicts, y_data)
        acc = acc_fn.accumulate()
        acc_fn.reset()
        # ÏÂÃæµÄ·´Ïò´«²¥¡¢´òÓ¡ÑµÁ·ÐÅÏ¢¡¢¸üÐÂ²ÎÊý¡¢ÌÝ¶ÈÇåÁã¶¼±»·â×°µ½ Model.fit() ÖÐ
        # ·´Ïò´«²¥ 
        loss.backward()
        # print("[Training] Backward done!")
        # if (batch_id+1) % 900 == 0:
        print("[Train] epoch: {}, batch_id: {}, loss is: {}, acc is: {:.4f}%".format(epoch, batch_id+1, loss.numpy(), 100*acc))
        losslist.append(loss.numpy())
        acclist.append(acc)
        # ¸üÐÂ²ÎÊý 
        optim.step()
        # ÌÝ¶ÈÇåÁã
        optim.clear_grad()

# save
# rtsr3.save("rtsr3.pdparams")
paddle.jit.save(
    layer=rtsr3,
    path="jit.save/rtsr3",
    input_spec=[paddle.static.InputSpec(shape=[None, 3, 180, 390], dtype='float32')])
# paddle.onnx.export(rtsr3, "onnx.save/rtsr3", input_spec=[paddle.static.InputSpec(shape=[None, 3, 180, 390], dtype='float32')], opset_version=11)
print("[Model Save] Done!")

# Visualize the loss and acc of training
l = np.array(losslist)
x_axis = len(l)
step = np.linspace(1,x_axis,x_axis)
plt.plot(step,l,label="Train Loss")
plt.legend(loc='upper right')
plt.title('step-loss')
plt.xlim((0, x_axis))
plt.gca().set_ylim(bottom=0)
plt.xlabel('step')
plt.ylabel('loss')
plt.savefig('doc/loss.png', bbox_inches='tight')
plt.show()

a = np.array(acclist)
plt.plot(step,a,label="Train Acc")
plt.legend(loc='upper right')
plt.title('step-acc')
plt.xlim((0, x_axis))
plt.gca().set_ylim(bottom=0)
plt.xlabel('step')
plt.ylabel('acc')
plt.savefig('doc/acc.png', bbox_inches='tight')
plt.show()
print("[Visualization] Done!")